In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from random import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
#HyperParameters
epochs = 5
step_size = 4
IMG_SIZE_ALEXNET = 227 # image size
validating_size = 89 # while cross validating, we are evaluating batch by batch
nodes_fc1 = 4096 # no of nodes on fc layer 1
nodes_fc2 = 4096 # no of nodes on fc layer 2
output_locations = 4 # minx, miny, maxx, maxy

In [3]:
X_test = np.load('object_localization_x_test.npy')

In [4]:
#X_test = X_test[:1000]

In [5]:
len(X_test)

12815

In [6]:
#Resetting graph
tf.reset_default_graph()

In [7]:
#Defining Placeholders
x = tf.placeholder(tf.float32,shape=[None,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3])
y_true_2 = tf.placeholder(tf.float32,shape=[None,output_locations])

In [8]:
##CONVOLUTION LAYER 1
#Weights for layer 1
w_1 = tf.Variable(tf.truncated_normal([11,11,3,96], stddev=0.01))
#Bias for layer 1
b_1 = tf.Variable(tf.constant(0.0, shape=[[11,11,3,96][3]]))
#Applying convolution
c_1 = tf.nn.conv2d(x, w_1,strides=[1, 4, 4, 1], padding='VALID')
#Adding bias
c_1 = c_1 + b_1
#Applying RELU
c_1 = tf.nn.relu(c_1)
print(c_1)

Tensor("Relu:0", shape=(?, 55, 55, 96), dtype=float32)


In [9]:
##POOLING LAYER1
p_1 = tf.nn.max_pool(c_1, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_1)

Tensor("MaxPool:0", shape=(?, 27, 27, 96), dtype=float32)


In [10]:
##CONVOLUTION LAYER 2
#Weights for layer 2
w_2 = tf.Variable(tf.truncated_normal([5,5,96,256], stddev=0.01))
#Bias for layer 2
b_2 = tf.Variable(tf.constant(1.0, shape=[[5,5,96,256][3]]))
#Applying convolution
c_2 = tf.nn.conv2d(p_1, w_2,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_2 = c_2 + b_2
#Applying RELU
c_2 = tf.nn.relu(c_2)
print(c_2)

Tensor("Relu_1:0", shape=(?, 27, 27, 256), dtype=float32)


In [11]:
##POOLING LAYER2
p_2 = tf.nn.max_pool(c_2, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_2)

Tensor("MaxPool_1:0", shape=(?, 13, 13, 256), dtype=float32)


In [12]:
##CONVOLUTION LAYER 3
#Weights for layer 3
w_3 = tf.Variable(tf.truncated_normal([3, 3, 256, 384], stddev=0.01))
#Bias for layer 3
b_3 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 256, 384][3]]))
#Applying convolution
c_3 = tf.nn.conv2d(p_2, w_3,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_3 = c_3 + b_3
#Applying RELU
c_3 = tf.nn.relu(c_3)
print(c_3)

Tensor("Relu_2:0", shape=(?, 13, 13, 384), dtype=float32)


In [13]:
##CONVOLUTION LAYER 4
#Weights for layer 4
w_4 = tf.Variable(tf.truncated_normal([3, 3, 384, 384], stddev=0.01))
#Bias for layer 4
b_4 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 384, 384][3]]))
#Applying convolution
c_4 = tf.nn.conv2d(c_3, w_4,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_4 = c_4 + b_4
#Applying RELU
c_4 = tf.nn.relu(c_4)
print(c_4)

Tensor("Relu_3:0", shape=(?, 13, 13, 384), dtype=float32)


In [14]:
##CONVOLUTION LAYER 5
#Weights for layer 5
w_5 = tf.Variable(tf.truncated_normal([3, 3, 384, 256], stddev=0.01))
#Bias for layer 5
b_5 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 384, 256][3]]))
#Applying convolution
c_5 = tf.nn.conv2d(c_4, w_5,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_5 = c_5 + b_5
#Applying RELU
c_5 = tf.nn.relu(c_5)
print(c_5)

Tensor("Relu_4:0", shape=(?, 13, 13, 256), dtype=float32)


In [15]:
##POOLING LAYER3
p_3 = tf.nn.max_pool(c_5, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_3)

Tensor("MaxPool_2:0", shape=(?, 6, 6, 256), dtype=float32)


In [16]:
#Flattening
flattened = tf.reshape(p_3,[-1,6*6*256])
print(flattened)

Tensor("Reshape:0", shape=(?, 9216), dtype=float32)


In [17]:
##Fully Connected Layer 1
#Getting input nodes in FC layer 1
input_size = int( flattened.get_shape()[1] )
#Weights for FC Layer 1
w1_fc = tf.Variable(tf.truncated_normal([input_size, nodes_fc1], stddev=0.01))
#Bias for FC Layer 1
b1_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc1] ) )
#Summing Matrix calculations and bias
s_fc1 = tf.matmul(flattened, w1_fc) + b1_fc
#Applying RELU
s_fc1 = tf.nn.relu(s_fc1)

In [18]:
#Dropout Layer 1
hold_prob1 = tf.placeholder(tf.float32)
s_fc1 = tf.nn.dropout(s_fc1,keep_prob=hold_prob1)
print(s_fc1)

Tensor("dropout/mul:0", shape=(?, 4096), dtype=float32)


In [19]:
##Fully Connected Layer 2
#Weights for FC Layer 2
w2_fc = tf.Variable(tf.truncated_normal([nodes_fc1, nodes_fc2], stddev=0.01))
#Bias for FC Layer 2
b2_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc2] ) )
#Summing Matrix calculations and bias
s_fc2 = tf.matmul(s_fc1, w2_fc) + b2_fc
#Applying RELU
s_fc2 = tf.nn.relu(s_fc2)
print(s_fc2)

Tensor("Relu_6:0", shape=(?, 4096), dtype=float32)


In [20]:
#Dropout Layer 2
hold_prob2 = tf.placeholder(tf.float32)
s_fc2 = tf.nn.dropout(s_fc2,keep_prob=hold_prob2)

In [21]:
##Fully Connected Layer 3 -- REGRESSION HEAD
#Weights for FC Layer 3
w3_fc_2 = tf.Variable(tf.truncated_normal([nodes_fc2,output_locations], stddev=0.01))
#Bias for FC Layer 3b3_fc = tf.Variable( tf.constant(1.0, shape=[output_classes] ) )
b3_fc_2 = tf.Variable( tf.constant(1.0, shape=[output_locations] ) )
#Summing Matrix calculations and bias
y_pred_2 = tf.matmul(s_fc2, w3_fc_2) + b3_fc_2
#Applying RELU
print(y_pred_2)

Tensor("add_7:0", shape=(?, 4), dtype=float32)


In [22]:
test_photos = X_test

In [23]:
from PIL import Image
import matplotlib.patches as patches
with tf.Session() as session:
    new_saver = tf.train.Saver()
    new_saver.restore(session, "CNN_OL.ckpt")
    print("Model restored.") 
    print('Initialized')
    steps = int(len(X_test)/1000)
    remaining = len(X_test)%1000
    l = []
    for i in range(steps):
        temp = session.run([y_pred_2], feed_dict={x: test_photos[i*1000:(i+1)*1000], hold_prob1:1,hold_prob2:1})
        l.append(temp[0])
        print("Step ",i)
    temp = session.run([y_pred_2], feed_dict={x: test_photos[steps*1000:], hold_prob1:1,hold_prob2:1})
    l.append(temp[0])
print(len(l))

INFO:tensorflow:Restoring parameters from CNN_OL.ckpt
Model restored.
Initialized
Step  0
Step  1
Step  2
Step  3
Step  4
Step  5
Step  6
Step  7
Step  8
Step  9
Step  10
Step  11
13


In [24]:
l[12].shape
m = []
steps = len(l)
for i in range(steps):
    for j in range(l[i].shape[0]):
        m.append(l[i][j])
len(m)

12815

In [25]:
l = pd.DataFrame(m)
l.columns = ['x1','x2','y1','y2']
l.shape

(12815, 4)

In [27]:
l['x1'] *= (640/227)
l['x2'] *= (640/227)
l['y1'] *= (480/227)
l['y2'] *= (480/227)

In [28]:
l

,x1,x2,y1,y2
0,237.832182,461.488075,79.329831,401.594367
1,68.251195,576.832152,131.160335,344.908205
2,134.632273,491.391537,47.063462,417.364088
3,231.443699,453.119063,94.563424,414.497705
4,117.555797,510.682303,34.901048,421.009548
5,174.582573,369.881952,17.550863,469.695276
6,127.408990,522.757090,90.146323,372.226896
7,68.900625,448.911154,40.317840,406.106473
8,219.471559,497.780063,141.812620,371.768277
9,11.559146,620.927777,-6.129024,466.533214


In [32]:
test=pd.read_csv('test.csv')
names = pd.DataFrame(test['image_name'])
y_pred = pd.concat([names, l], axis=1, sort=False)
y_pred

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,237.832182,461.488075,79.329831,401.594367
1,1473231475010DeeplearnS11276.png,68.251195,576.832152,131.160335,344.908205
2,JPEG_20161205_135307_1000155917326.png,134.632273,491.391537,47.063462,417.364088
3,JPEG_20160711_123440_1000518778437.png,231.443699,453.119063,94.563424,414.497705
4,JPEG_20160803_115329_100034020722.png,117.555797,510.682303,34.901048,421.009548
5,147444974116511473239803010-Mast--Harbour-Men-...,174.582573,369.881952,17.550863,469.695276
6,JPEG_20160622_110649_1000527459853.png,127.408990,522.757090,90.146323,372.226896
7,JPEG_20160823_120737_1000784898268.png,68.900625,448.911154,40.317840,406.106473
8,1480965956645IMG_3094.png,219.471559,497.780063,141.812620,371.768277
9,14732348976417a565e40d545452688130062b267a2d2.png,11.559146,620.927777,-6.129024,466.533214


In [33]:
y_pred.to_csv('y_pred.csv',index=False)